In [7]:
import time
import os
import paramiko
from io import StringIO
import time
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from IPython.display import clear_output
import datetime

In [8]:
# SSH credentials and remote CSV file path
# host = 'lxslc7.ihep.ac.cn'
# username = os.environ.get('IHEP_USERNAME')
# password = os.environ.get('IHEP_PASSWORD')
# remote_csv_file = '/hpcfs/cepc/higgs/frank/GNN/ds/output_tag/summaries_0.csv'
# remote_csv_file = '/hpcfs/cepc/higgs/frank/GNN/ds/output_rec/summaries_0.csv'


host = 'bl-0.inpac.sjtu.edu.cn'
username = os.environ.get('SJTU_USERNAME')
password = os.environ.get('SJTU_PASSWORD')
remote_csv_file = '/lustre/collider/zhangyulei/DeepLearning/Tracking/workspace/output/summaries_0.csv'

# Establish an SSH connection
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, username=username, password=password)
print(f'Successfully connected to {username}@{host}')

Successfully connected to zhangyulei@bl-0.inpac.sjtu.edu.cn


In [9]:
def update_csv(remote_file: str = None):
    sftp = ssh.open_sftp()

    if remote_file is None:
        remote_file = remote_csv_file

    with sftp.open(remote_file, 'r') as file:
        remote_csv_data = file.read().decode('utf-8')
        timestamp = sftp.stat(remote_file).st_mtime

        dt_object = datetime.datetime.fromtimestamp(timestamp)

        return pd.read_csv(StringIO(remote_csv_data)), \
            f"{dt_object.year} - {dt_object.month} - {dt_object.day}   {dt_object.hour} : {dt_object.minute}"


def create_trace(x, y, **kwargs):
    return go.Scatter(x=x, y=y, **kwargs)


In [10]:
def visual_link(df, t):
    y_max, y_min = df.max()[['train_loss', 'valid_loss']].max(), df.min()[['train_loss', 'valid_loss']].min()

    df_new = df[['epoch', 'train_loss', 'valid_loss', 'valid_acc']]. \
        groupby('epoch').transform("mean").drop_duplicates(keep='last', subset=['train_loss'])

    fig = go.Figure()

    traces = [
        (
            df.index, df['train_loss'],
            {
                'mode': 'lines', 'name': "itr: train", 'line': {'dash': 'dot'}, 'legendgroup': "Itr Loss"
            }),
        (
            df.index, df['valid_loss'],
            {
                'mode': 'lines', 'name': "itr: valid", 'line': {'dash': 'dot'}, 'legendgroup': "Itr Loss"
            }),
        (
            df_new.index, -np.log10(1 - df_new['valid_acc']),
            {
                'mode': 'lines', 'name': "itr: accuracy", 'line': {'dash': 'dot', 'color': "#11ADF0"}, 'yaxis': 'y2',
                'legendgroup': "Edge Label"
            }),
        (
            df_new.index, df_new['train_loss'],
            {'mode': 'lines+markers', 'name': "epoch: train", 'legendgroup': "Epoch Loss"}),
        (
            df_new.index, df_new['valid_loss'],
            {'mode': 'lines+markers', 'name': "epoch: valid", 'legendgroup': "Epoch Loss"})
    ]

    for x, y, kwargs in traces:
        fig.add_trace(create_trace(x, y, **kwargs))

    for lr in df.drop_duplicates(keep="first", subset=['lr'])['lr'].items():
        print(lr)
        fig.add_vline(x=lr[0], line_width=2, line_dash="dash", line_color="grey")
        fig.add_annotation(
            text=f'$\eta = {lr[1]}$',
            x=lr[0] + 0.5, y=np.log10(y_min) * 1.02,  # Set the position using numeric values
            xanchor='left',
            # yanchor='bottom',
            showarrow=False,
            font=dict(size=14, color='grey')
        )

    fig.update_layout(
        title=f"{t} --> Epoch: {df['epoch'].iloc[-1]}, Iteration: {df['itr'].iloc[-1]}",
        width=1200,
        height=700,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(
            title_text="Total iteration",
            showgrid=False,
            mirror=True,
            showline=True,
            zeroline=False,
            linewidth=2,
            linecolor='#666666', gridcolor='#d9d9d9',
            domain=[0.1, 0.85],
        ),
        yaxis=dict(
            title=r"Loss",
            titlefont=dict(color="#d62728"),
            tickfont=dict(color="#d62728"),
            showgrid=False,
            linecolor="#d62728", gridcolor='#d9d9d9',
            zeroline=False,
            type="log",
            range=[np.log10(y_min) * 1.05, np.log10(y_max) * 1.05],
        ),
        yaxis2=dict(
            # title=r"$\textrm{95% CI }{\Large \kappa_{2V}}\textrm{ Interval}$",
            title=r"$-log(\epsilon_{error})$",
            titlefont=dict(color="#11ADF0"),
            tickfont=dict(color="#11ADF0"),
            side="right",
            showgrid=False,
            linecolor="#11ADF0", gridcolor='#d9d9d9',
            zeroline=False,
            # type="log",
            # range=[3.0, 3.7],
        ),
    )

    return fig


def visual_momentum(df, t):
    y_max, y_min = df.max()[['train_loss', 'valid_loss']].max(), df.min()[['train_loss', 'valid_loss']].min()

    df_new = df[['epoch', 'train_loss', 'valid_loss', 'valid_dp_mean', 'valid_dp_std']]. \
        groupby('epoch').transform("mean").drop_duplicates(keep='last', subset=['train_loss'])

    fig = go.Figure()

    traces = [
        (
            df.index, df['train_loss'],
            {
                'mode': 'lines', 'name': "itr: train", 'line': {'dash': 'dot'}, 'legendgroup': "Itr Loss"
            }),
        (
            df.index, df['valid_loss'],
            {
                'mode': 'lines', 'name': "itr: valid", 'line': {'dash': 'dot'}, 'legendgroup': "Itr Loss"
            }),
        (
            df_new.index, df_new['valid_dp_mean'],
            {
                'mode': 'lines', 'name': "p: mean", 'line': {'dash': 'dot', 'color': "#ff7f0e"}, 'yaxis': 'y2',
                'legendgroup': "Edge Label"
            }),
        (
            df_new.index, df_new['valid_dp_std'],
            {
                'mode': 'lines', 'name': "p: std", 'line': {'dash': 'dot', 'color': "#64a05f"}, 'yaxis': 'y3',
                'legendgroup': "Edge Label"
            }),
        (
            df_new.index, df_new['train_loss'],
            {'mode': 'lines+markers', 'name': "epoch: train", 'legendgroup': "Epoch Loss"}),
        (
            df_new.index, df_new['valid_loss'],
            {'mode': 'lines+markers', 'name': "epoch: valid", 'legendgroup': "Epoch Loss"})
    ]

    for x, y, kwargs in traces:
        fig.add_trace(create_trace(x, y, **kwargs))

    for lr in df.drop_duplicates(keep="first", subset=['lr'])['lr'].items():
        print(lr)
        fig.add_vline(x=lr[0], line_width=2, line_dash="dash", line_color="grey")
        fig.add_annotation(
            text=f'$\eta = {lr[1]:.3e}$',
            x=lr[0] + 0.5, y=np.log10(y_min) * 1.02,  # Set the position using numeric values
            xanchor='left',
            # yanchor='bottom',
            showarrow=False,
            font=dict(size=14, color='grey')
        )

    fig.update_layout(
        title=f"{t} --> Epoch: {df['epoch'].iloc[-1]}, Iteration: {df['itr'].iloc[-1]}",
        width=1200,
        height=700,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(
            title_text="Total iteration",
            showgrid=False,
            mirror=True,
            showline=True,
            zeroline=False,
            linewidth=2,
            linecolor='#666666', gridcolor='#d9d9d9',
            domain=[0.1, 0.85],
        ),
        yaxis=dict(
            title=r"Loss",
            titlefont=dict(color="#d62728"),
            tickfont=dict(color="#d62728"),
            showgrid=False,
            linecolor="#d62728", gridcolor='#d9d9d9',
            zeroline=False,
            type="log",
            range=[np.log10(y_min) * 1.05, np.log10(y_max) * 1.05],
        ),
        yaxis2=dict(
            title=r"$\mu^{p}_{\textrm{valid}}$",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            anchor="x",
            overlaying="y",
            side="right",
            showgrid=False,
            linecolor="#ff7f0e", gridcolor='#d9d9d9',
            zeroline=False,
            # range=[-mu_max * 1.05, mu_max * 1.05]
        ),
        yaxis3=dict(
            title=r"$\sigma^{p}_{\textrm{valid}}$",
            titlefont=dict(color="#64a05f"),
            tickfont=dict(color="#64a05f"),
            anchor="free",
            overlaying="y",
            side="right",
            position=0.95,
            showgrid=False,
            linecolor="#64a05f", gridcolor='#d9d9d9',
            zeroline=False,
            # range=[0.0, std_max],
            # range=[0.0, 1.0],
        ),
    )

    return fig

In [11]:
tag = '/hpcfs/cepc/higgs/frank/GNN/ds/output_tag/summaries_0.csv'
rec = '/hpcfs/cepc/higgs/frank/GNN/ds/output_rec/summaries_0.csv'

# tar = tag
tar = rec

# df, t = update_csv('/lustre/collider/zhangyulei/DeepLearning/Tracking/workspace/output.rec.momentum/summaries_0.csv')
df, t = update_csv('/lustre/collider/zhangyulei/DeepLearning/Tracking/workspace/output.rec.momentum.beta0.1/summaries_0.csv')
# fig = visual_link(df, t)
fig = visual_momentum(df, t)
fig.show()
#
# while True:
#     df, t = update_csv(tar)
#     fig = update_fig(df, t)
#     time.sleep(120)
#     clear_output(wait=False)
#     fig.show()

(0, 0.002)
(18, 0.0028284271247461)


In [9]:
def update_loss(df, t):
    y_max, y_min = df.max()[['train_loss', 'valid_loss']].max(), df.min()[['train_loss', 'valid_loss']].min()

    # df_new = df[['epoch', 'train_loss', 'valid_loss', 'train_y_loss', 'train_p_loss', 'train_y_weight', 'train_p_weight']]. \
    #     groupby('epoch').transform("mean").drop_duplicates(keep='last', subset=['train_loss'])

    fig = go.Figure()

    traces = [
        (
            df.index, df['train_y_loss'],
            {
                'mode': 'lines', 'name': "y loss", 'line': {'dash': 'dot', 'color': "#ff7f0e"},
                'legendgroup': "Sep. Loss"
            }),
        (
            df.index, df['train_p_loss'],
            {
                'mode': 'lines', 'name': "p loss", 'line': {'dash': 'dot', 'color': "#64a05f"},
                'legendgroup': "Sep. Loss"
            }),
        (
            df.index, df['train_loss'],
            {
                'mode': 'lines', 'name': "train loss", 'line': {'dash': 'dashdot'}, 'legendgroup': "Total Loss"
            }),
        (
            df.index, df['valid_loss'],
            {
                'mode': 'lines', 'name': "valid loss", 'line': {'dash': 'dashdot'}, 'legendgroup': "Total Loss"
            }),
        (
            df.index, df['train_y_weight'],
            {'mode': 'lines', 'name': "y weight", 'legendgroup': "Loss weight", 'yaxis': 'y3'}),
        (
            df.index, df['train_p_weight'],
            {'mode': 'lines', 'name': "p weight", 'legendgroup': "Loss weight", 'yaxis': 'y3'})
    ]

    for x, y, kwargs in traces:
        fig.add_trace(create_trace(x, y, **kwargs))

    for lr in df.drop_duplicates(keep="first", subset=['lr'])['lr'].items():
        print(lr)
        fig.add_vline(x=lr[0], line_width=2, line_dash="dash", line_color="grey")
        fig.add_annotation(
            text=f'$\eta = {lr[1]}$',
            x=lr[0] + 0.5, y=np.log10(y_min) * 1.02,  # Set the position using numeric values
            xanchor='left',
            # yanchor='bottom',
            showarrow=False,
            font=dict(size=14, color='grey')
        )

    fig.update_layout(
        title=f"{t} --> Epoch: {df['epoch'].iloc[-1]}, Iteration: {df['itr'].iloc[-1]}",
        width=1200,
        height=700,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        xaxis=dict(
            title_text="Total iteration",
            showgrid=False,
            mirror=True,
            showline=True,
            zeroline=False,
            linewidth=2,
            linecolor='#666666', gridcolor='#d9d9d9',
            domain=[0.1, 0.85],
        ),
        yaxis=dict(
            title=r"Loss",
            titlefont=dict(color="#d62728"),
            tickfont=dict(color="#d62728"),
            showgrid=False,
            linecolor="#d62728", gridcolor='#d9d9d9',
            zeroline=False,
            type="log",
            # range=[np.log10(y_min) * 1.05, np.log10(y_max) * 1.05],
        ),
        # yaxis2=dict(
        #     # title=r"$\textrm{95% CI }{\Large \kappa_{2V}}\textrm{ Interval}$",
        #     title=r"$-log(\epsilon_{error})$",
        #     titlefont=dict(color="#11ADF0"),
        #     tickfont=dict(color="#11ADF0"),
        #     anchor="free",
        #     overlaying="y",
        #     side="left",
        #     position=0.0,
        #     showgrid=False,
        #     linecolor="#11ADF0", gridcolor='#d9d9d9',
        #     zeroline=False,
        #     # type="log",
        #     range=[0, 1],
        # ),
        yaxis3=dict(
            title=r"weight",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            anchor="x",
            overlaying="y",
            side="right",
            showgrid=False,
            linecolor="#ff7f0e", gridcolor='#d9d9d9',
            zeroline=False,
            type="log",
            # range=[0, 1],
        ),
        # yaxis4=dict(
        #     title=r"$\sigma^{p}_{\textrm{valid}}$",
        #     titlefont=dict(color="#64a05f"),
        #     tickfont=dict(color="#64a05f"),
        #     anchor="free",
        #     overlaying="y",
        #     side="right",
        #     position=0.95,
        #     showgrid=False,
        #     linecolor="#64a05f", gridcolor='#d9d9d9',
        #     zeroline=False,
        #     # range=[0.0, std_max],
        #     # range=[0.0, 1.0],
        # ),
    )

    return fig


fig = update_loss(df, t)
fig.show()

KeyError: 'train_y_loss'

In [10]:
df_new = df[[
    'epoch',
    # 'valid_num_tracks_diff_0',
    # 'valid_num_tracks_diff_1',
    # 'valid_num_tracks_diff_2',
    # 'valid_num_tracks_diff_-1',
    # 'valid_num_tracks_diff_-2',
    # 'valid_num_tracks_diff_overflow',
    # 'valid_num_tracks_diff_underflow',
    'valid_TP',
    'valid_TN',
    'valid_FP',
    'valid_FN',
]].groupby('epoch', as_index=False).transform("sum").drop_duplicates(keep='last')

In [11]:
def update_stack(df: pd.DataFrame, columns, t):
    fig = go.Figure()

    norm_sum = df[columns].sum(axis=1)

    for key in columns:
        fig.add_trace(go.Bar(
            name=key,
            y=df[key] / norm_sum,
            x=df['epoch'],
            width=1,
            offset=0,
        ))

    fig.update_layout(
        title=f"{t} --> Epoch: {df['epoch'].iloc[-1]}",
        width=800,
        height=700,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        barmode="stack",
        # yaxis=dict(
        #     type="log",
        # ),

    )

    fig.show()


update_stack(df_new, [col for col in df_new.columns if 'valid_num_tracks_diff' in col], t)

update_stack(df_new, [col for col in df_new.columns if 'valid_num_tracks_diff' not in col][1:], t)


In [129]:
df

,lr,train_loss,l1,l2,grad_norm,train_batches,train_y_loss,train_p_loss,train_y_weight,train_p_weight,...,valid_loss,valid_acc,valid_TP,valid_FP,valid_TN,valid_FN,valid_batches,valid_sum_total,valid_dp_mean,valid_dp_std
0,0.002000,0.765689,37770.705953,63.890343,0.823649,36,0.852184,0.055949,0.945929,0.054071,...,0.466937,0.914448,338432.962158,43967.838379,301626.980957,15913.734161,16,699941.532715,2.066267,5.777748
1,0.002000,0.384131,37328.772916,63.582687,0.741352,35,0.384424,0.036229,0.999161,0.000839,...,0.341082,0.942964,380514.859375,33318.140747,347328.982422,10706.114990,15,771868.105469,3.120670,7.772672
2,0.002000,0.380296,37265.796022,63.522645,0.651022,6,0.380577,0.034131,0.999189,0.000811,...,0.294844,0.946044,74117.425781,6181.250244,67894.181641,1918.097260,3,150110.953125,3.222010,7.490502
3,0.002000,0.304124,36892.501916,63.132024,1.257729,41,0.304344,0.038384,0.999172,0.000828,...,0.305424,0.945499,370634.666016,34323.481018,336562.504883,6441.177455,18,747961.822266,3.282121,8.277453
4,0.002000,0.281334,36610.043532,62.798194,0.787097,36,0.281534,0.037524,0.999182,0.000818,...,0.261839,0.955325,370941.292480,30033.775879,333979.461426,2931.075249,16,737885.597656,3.131330,7.825272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,0.002828,0.148198,30173.195124,59.593392,0.257953,21,0.148298,0.031493,0.999146,0.000854,...,0.138330,0.975628,173522.832031,6442.795502,161813.224609,1934.038452,9,343712.888672,2.718887,6.922829
104,0.002828,0.114547,30141.979043,59.652822,0.345984,30,0.114617,0.030918,0.999156,0.000844,...,0.121181,0.980152,255241.750000,7110.590424,241520.762695,2948.577755,13,506821.666016,2.736562,6.634753
105,0.002828,0.100144,30143.065358,59.714441,0.332051,30,0.100202,0.031011,0.999165,0.000835,...,0.113147,0.981791,263205.723633,6362.026520,250637.354980,3168.255966,13,523373.351562,2.580777,6.646007
106,0.002828,0.120493,29991.500449,59.661272,0.322493,41,0.120564,0.037917,0.999139,0.000861,...,0.114328,0.981707,356563.391602,10501.486511,342201.911621,2518.990513,18,711785.779297,3.233347,8.216982
